In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mplsoccer import Pitch, Sbopen, VerticalPitch
import json

In [14]:
class DataHandler:
    def __init__(self, country):
        self.country = country
        
    def get_teams(self, team_info):
        team_info = dict(team_info)
        teams = list(team_info.keys())
        return pd.Series([int(teams[0]),int(teams[1])])
    
    def shortNameRead(self, name):
            dict_name = '{"' + name + '": 0}'


            real_name = list(json.loads(dict_name, strict=False).keys())[0].replace("\t", " ")
            return real_name

    def roleRead(self, role):
        return role["name"]

    def load_data(self, country):
        path_matches = f"data_wyscout/matches/matches_{country}.json"
        path_events = f"data_wyscout/events/events_{country}.json"
        matches = pd.read_json(path_matches)

        matches[["home", "away"]] = matches.apply(lambda x: self.get_teams(x["teamsData"]), axis=1)
        teams = pd.read_json("data_wyscout/teams.json")
        
        matches = pd.merge(matches, teams[["wyId", "officialName"]].set_index("wyId"), left_on="home", right_index=True)
        matches = pd.merge(matches, teams[["wyId", "officialName"]].set_index("wyId"), left_on="away", right_index=True, suffixes=["_home", "_away"])
        
        events = pd.read_json(path_events)
        events = pd.merge(events,  teams[["wyId", "officialName"]].set_index("wyId"), left_on="teamId", right_index=True)
        events = pd.merge(events, matches[["wyId", 'officialName_home','officialName_away']].set_index("wyId"), left_on="matchId", right_index=True)
        
        players = pd.read_json("data_wyscout/players.json")

        players["roleName"] = players.role.apply(lambda x: self.roleRead(x))    
        players["shortName"] = players.shortName.apply(lambda x: self.shortNameRead(x))
        
        events = pd.merge(events,  players[["wyId", "shortName", "roleName"]].set_index("wyId"), left_on="playerId", right_index=True)
        
        return events, matches
    
    def save_data(self, how="Full"):
        events, matches = self.load_data(self.country)
        
        if how == "Full":
            events.to_csv(f"data_wyscout/{self.country}_17_18.csv", index=False)
            matches.to_csv(f"data_wyscout/matches_{self.country}_17_18.csv", index=False)
        elif how == "Events":
            events.to_csv(f"data_wyscout/{self.country}_17_18.csv", index=False)
        elif how == "Matches":
            matches.to_csv(f"data_wyscout/matches_{self.country}_17_18.csv", index=False)
        else:
            print("No data to save")

In [9]:
dth = DataHandler("England")

In [11]:
dth.save_data()

In [12]:
dth = DataHandler("Spain")

In [13]:
dth.save_data()